In [1]:
import sys
from tqdm import tqdm
import pandas as pd
import os
import datetime
import OptionTools
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np

In [2]:
def OLS_evaluate(df, add=False):
    df = df.dropna()
    MON = df['week_day'].apply(lambda x: 1 if x==0 else 0).to_numpy()
    TUE = df['week_day'].apply(lambda x: 1 if x==1 else 0).to_numpy()
    WED = df['week_day'].apply(lambda x: 1 if x==2 else 0).to_numpy()
    THU = df['week_day'].apply(lambda x: 1 if x==3 else 0).to_numpy()
    FRI = df['week_day'].apply(lambda x: 1 if x==4 else 0).to_numpy()
    y = df.imp_change.values
    X = np.column_stack([MON, TUE, WED, THU, FRI])
    if add:
        X = np.column_stack([MON, TUE, WED, THU, FRI, df['to_maturity'].values/252])
    model = sm.OLS(y, X)
    model = model.fit()
    for j, k in enumerate(model.params):
        print(round(k, 4))
        print("({})".format(round(model.tvalues[j], 2)))
#     print(round(model.f_pvalue, 4))
    print(round(model.rsquared_adj, 4))
    print(model.summary())
    return model

def quadratic_OLS(sel):
    mats = np.log(sel['to_maturity'].values)
    mats_sq = np.log(sel['to_maturity'].values) ** 2
    X = np.column_stack([mats, mats_sq])
    X = sm.add_constant(X)
    y = sel.imp_change.values
    model = sm.OLS(y, X)
    model = model.fit()
    print(model.summary())

def abs_OLS(df, add=False):
    df = df.dropna()
    MON = df['week_day'].apply(lambda x: 1 if x==0 else 0).to_numpy()
    TUE = df['week_day'].apply(lambda x: 1 if x==1 else 0).to_numpy()
    WED = df['week_day'].apply(lambda x: 1 if x==2 else 0).to_numpy()
    THU = df['week_day'].apply(lambda x: 1 if x==3 else 0).to_numpy()
    FRI = df['week_day'].apply(lambda x: 1 if x==4 else 0).to_numpy()
    y = np.abs(df.imp_change.values)
    X = np.column_stack([MON, TUE, WED, THU, FRI])
    if add:
        X = sm.add_constant(X)
    model = sm.OLS(y, X)
    model = model.fit()
    print(model.pvalues)
    print(model.params)
    print(model.summary())
    return model

def cp(df, days=6):
    calls= df[df['type']=='C']
    puts= df[df['type']=='P']
    calls=calls[calls.to_maturity>=days].dropna()
    puts=puts[puts.to_maturity>=days].dropna()
    return calls, puts

In [3]:
def weekly_OLS_evaluate(df):
    a1 = df[(df['to_maturity']<= 29) & (df['to_maturity']>= 6)]
    a2 = df[(df['to_maturity']<= 51) & (df['to_maturity']>= 30)]
    a3 = df[(df['to_maturity']<= 106) & (df['to_maturity']>= 52)]
    a4 = df[(df['to_maturity']<= 175) & (df['to_maturity']>= 107)]
    a5 = df[df['to_maturity'] >= 176]
    for k in [a1, a2, a3, a4, a5]:
        print(k.to_maturity.min(), k.to_maturity.max())
        OLS_evaluate(k)
        


In [4]:
def td_OLS_evaluate(df):
    a1 = df[(df['to_maturity']<= 22) & (df['to_maturity']>= 6)]
    a2 = df[(df['to_maturity']<= 37) & (df['to_maturity']>= 23)]
    a3 = df[(df['to_maturity']<= 73) & (df['to_maturity']>= 38)]
    a4 = df[(df['to_maturity']<= 119) & (df['to_maturity']>= 74)]
    a5 = df[df['to_maturity'] >= 120]
    for k in [a1, a2, a3, a4, a5]:
        print(k.to_maturity.min(), k.to_maturity.max())
        OLS_evaluate(k)

In [12]:
df = pd.DataFrame()
for fn in os.listdir('td_vol/call'):
    path = 'td_vol/call/'+fn
    tmp = pd.read_csv(path)
    tmp['type'] = 'C'
    df = pd.concat([df, tmp])
    
for fn in os.listdir('td_vol/put'):
    path = 'td_vol/put/'+fn
    tmp = pd.read_csv(path)
    tmp['type'] = 'P'
    df = pd.concat([df, tmp])
    
calls, puts = cp(df,6)

In [13]:
def describe(df, col_name):
    target = df[col_name]
    tl = [target.mean(), target.std(), target.max()-target.min(), np.percentile(target, 25), target.median(), np.percentile(target, 75)]
    ttl = [round(x, 3) for x in tl]
    for k in ttl:
        print(k, end='\t')

# 使用通过implied futures price计算的vol

In [14]:
np.percentile(calls.to_maturity, 80)

118.0

In [15]:
calls['abs_ic'] = calls.imp_change.abs()
describe(calls[(calls['to_maturity'] <= 37)&(calls['to_maturity'] >= 23)], 'imp_change')
print()
describe(calls[(calls['to_maturity'] <= 37)&(calls['to_maturity'] >= 23)], 'abs_ic')

-0.007	0.058	0.518	-0.033	-0.007	0.024	
0.042	0.041	0.285	0.015	0.029	0.053	

In [16]:
OLS_evaluate(calls)

-0.0038
(-1.19)
-0.0109
(-3.47)
-0.0017
(-0.55)
0.007
(2.13)
0.0067
(2.06)
0.0103
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     5.575
Date:                Tue, 07 Mar 2023   Prob (F-statistic):           0.000185
Time:                        22:43:16   Log-Likelihood:                 2450.4
No. Observations:                1760   AIC:                            -4891.
Df Residuals:                    1755   BIC:                            -4863.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------

In [17]:
calls['year'] = calls['date'].apply(lambda x: x // 10000)

In [18]:
OLS_evaluate(calls[calls['year']<=2017])
OLS_evaluate(calls[calls['year']==2018])
OLS_evaluate(calls[calls['year']==2019])
OLS_evaluate(calls[calls['year']>=2020])

-0.0072
(-0.75)
-0.003
(-0.34)
-0.0065
(-0.71)
0.0059
(0.62)
0.0039
(0.42)
-0.0094
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.4045
Date:                Tue, 07 Mar 2023   Prob (F-statistic):              0.805
Time:                        22:43:16   Log-Likelihood:                 332.19
No. Observations:                 256   AIC:                            -654.4
Df Residuals:                     251   BIC:                            -636.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
a1 = calls[(calls['to_maturity']<= 22) & (calls['to_maturity']>= 6)]
a2 = calls[(calls['to_maturity']<= 37) & (calls['to_maturity']>= 23)]
a3 = calls[(calls['to_maturity']<= 73) & (calls['to_maturity']>= 38)]
a4 = calls[(calls['to_maturity']<= 119) & (calls['to_maturity']>= 74)]
a5 = calls[calls['to_maturity'] >= 120]
OLS_evaluate(a1)
OLS_evaluate(a2)
OLS_evaluate(a3)
OLS_evaluate(a4)
OLS_evaluate(a5)

In [19]:
def weekly_basics(df):
    a1 = df[(df['to_maturity']<= 31) & (df['to_maturity']>= 8)]
    a2 = df[(df['to_maturity']<= 55) & (df['to_maturity']>= 32)]
    a3 = df[(df['to_maturity']<= 108) & (df['to_maturity']>= 56)]
    a4 = df[(df['to_maturity']<= 177) & (df['to_maturity']>= 109)]
    a5 = df[df['to_maturity'] >= 178]
    for k in [a1, a2, a3, a4, a5]:
        print(round(k.imp_change.mean(), 4), end='\t')
        print(round(k.imp_change.std(), 4), end='\t')
        print(round(k.imp_change.max() - k.imp_change.min(), 4), end='\t')
        print(round(np.percentile(k.imp_change, 25), 4), end='\t')
        print(round(k.imp_change.median(), 4), end='\t')
        print(round(np.percentile(k.imp_change, 75), 4), end='\t')
        print()

In [20]:
def wd_weekly_basics(df, ab=False):
    tdf = df.loc[:]
    if ab:
        tdf['imp_change'] = np.abs(tdf.imp_change)
    a1 = tdf[(tdf['to_maturity']<= 31) & (tdf['to_maturity']>= 8)]
    a2 = tdf[(tdf['to_maturity']<= 55) & (tdf['to_maturity']>= 32)]
    a3 = tdf[(tdf['to_maturity']<= 108) & (tdf['to_maturity']>= 56)]
    a4 = tdf[(tdf['to_maturity']<= 177) & (tdf['to_maturity']>= 109)]
    a5 = tdf[tdf['to_maturity'] >= 178]
    for k in [a1, a2, a3, a4, a5]:
        s = [k[k['week_day']==x] for x in range(5)]
        for sb in s:
            print(round(sb.imp_change.mean(), 4), end='\t')
        print()

In [21]:
def yearlys(df):
    c1 = calls[(calls['date']<20160101)]
    c2 = calls[(calls['date']<20170101)&(calls['date']>=20160101)]
    c3 = calls[(calls['date']<20180101)&(calls['date']>=20170101)]
    c4 = calls[(calls['date']<20190101)&(calls['date']>=20180101)]
    c5 = calls[(calls['date']<20200101)&(calls['date']>=20190101)]
    c6 = calls[(calls['date']>=20200101)]
    for k in [c1,c2,c3,c4,c5,c6]:
        s = [k[k['week_day']==x] for x in range(5)]
        for sb in s:
            print(round(sb.imp_change.mean(), 4), end='\t')
        print()

In [22]:
def bymat(df):
    a1 = df[(df['to_maturity']<= 31) & (df['to_maturity']>= 8)]
    a2 = df[(df['to_maturity']<= 55) & (df['to_maturity']>= 32)]
    a3 = df[(df['to_maturity']<= 108) & (df['to_maturity']>= 56)]
    a4 = df[(df['to_maturity']<= 177) & (df['to_maturity']>= 109)]
    a5 = df[df['to_maturity'] >= 178]
    for k in [a1, a2, a3, a4, a5]:
        OLS_evaluate(k)

# 按照交易日期划分

In [23]:
c1 = calls[(calls['date']<20160101)]
c2 = calls[(calls['date']<20170101)&(calls['date']>=20160101)]
c3 = calls[(calls['date']<20180101)&(calls['date']>=20170101)]
c4 = calls[(calls['date']<20190101)&(calls['date']>=20180101)]
c5 = calls[(calls['date']<20200101)&(calls['date']>=20190101)]
c6 = calls[(calls['date']>=20200101)]
for k in [c1, c2, c3, c4, c5, c6]:
    OLS_evaluate(k)

-0.0072
(-0.75)
-0.003
(-0.34)
-0.0065
(-0.71)
0.0059
(0.62)
0.0039
(0.42)
-0.0094
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.4045
Date:                Tue, 07 Mar 2023   Prob (F-statistic):              0.805
Time:                        22:43:17   Log-Likelihood:                 332.19
No. Observations:                 256   AIC:                            -654.4
Df Residuals:                     251   BIC:                            -636.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------

ValueError: zero-size array to reduction operation maximum which has no identity

# 按照到期日区间划分

In [24]:
a1 = calls[(calls['to_maturity']<= 22) & (calls['to_maturity']>= 6)]
a2 = calls[(calls['to_maturity']<= 37) & (calls['to_maturity']>= 23)]
a3 = calls[(calls['to_maturity']<= 73) & (calls['to_maturity']>= 38)]
a4 = calls[(calls['to_maturity']<= 119) & (calls['to_maturity']>= 74)]
a5 = calls[calls['to_maturity'] >= 120]

for k in [a1, a2, a3, a4, a5]:
    OLS_evaluate(k)

-0.0204
(-2.16)
-0.0064
(-0.74)
0.0009
(0.11)
0.004
(0.4)
0.003
(0.3)
0.0014
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.134
Date:                Tue, 07 Mar 2023   Prob (F-statistic):              0.340
Time:                        22:43:18   Log-Likelihood:                 419.64
No. Observations:                 383   AIC:                            -829.3
Df Residuals:                     378   BIC:                            -809.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------

# 到期日加权method?
# 可以思考一下：具体如何加权？

In [25]:
# calls['x'] = calls.to_maturity.apply(lambda x: 1/x)
calls['x'] = calls.to_maturity.apply(lambda x: 1/np.log(x))
tmp1 = calls.groupby('date').x.sum().reset_index()
tmp1.columns = ['date', 'sum_mat']

tmp_call = calls.merge(tmp1, on='date')
tmp_call['w'] = tmp_call['x'] / tmp_call['sum_mat']
tmp_call['weighted_iv'] = tmp_call.imp_vol * tmp_call.w
weighted_ivs = tmp_call.groupby('date').sum().reset_index()[['date', 'weighted_iv']]

calendar = calls[['date', 'week_day']]
weighted_ivs = weighted_ivs.merge(calendar, on='date').drop_duplicates()
weighted_ivs['imp_change'] = weighted_ivs.weighted_iv / weighted_ivs.weighted_iv.shift(1) - 1
weighted_ivs = weighted_ivs.dropna()
OLS_evaluate(weighted_ivs[weighted_ivs['date']>=20190101])

-0.0043
(-0.71)
-0.0142
(-2.37)
-0.0012
(-0.2)
0.0182
(3.0)
0.0054
(0.88)
0.0243
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     3.968
Date:                Tue, 07 Mar 2023   Prob (F-statistic):            0.00354
Time:                        22:43:18   Log-Likelihood:                 675.85
No. Observations:                 478   AIC:                            -1342.
Df Residuals:                     473   BIC:                            -1321.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------